# Storing Data pada MongoDB Atlas

## Load Dataset

In [1]:
import pandas as pd
from haystack import Document
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from pymongo import MongoClient
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack import Document
from haystack.components.preprocessors import DocumentSplitter
import os
from getpass import getpass

d:\Enviroments\anaconda3\envs\day45-haystack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle('../data/datasets.pkl')

In [3]:
df.head()

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1,gender,material,category
69,Buxton Heiress Pik-Me-Up Framed Case,Buxton,"['Leather', 'Imported', 'synthetic lining', 'F...","43,930inClothing,Shoesamp;Jewelry(",5 star,B00007GDFV,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,['Authentic crunch leather with rich floral em...,16.95,"[B07C9V84JD, B01J6JE05G, B07J11WZ5Y, B07JJQFHS...","[B07C9V84JD, B01J6JE05G, B07JJQFHS5, B003EGITU...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN,female,Leather,Accessories
352,Disguise Tiny Treats Pink Leopard,NaN,"['polyester', 'You can return this item for an...","5,843,505inClothing,ShoesJewelry(",5 star,B0002C6NL6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,['A grrreat pink jumpsuit with attached tail a...,25.99,NaN,NaN,NaN,NaN,NaN,NaN,female,Polyester,Dresses/Jumpsuits
410,Dream PJ's Blue - Large - Part #: 25BLG,Ethical/Spot,['Product Dimensions:\n \n8...,"17,183,425inClothing,Shoesamp;Jewelry(",5 star,B0002TKBSU,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,"['SOFT AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT...",15.99,NaN,NaN,NaN,NaN,NaN,NaN,unisex,Unknown,Undergarments/Sleepwear
512,Buxton Heiress Pik-Me-Up Framed Case,Buxton,"['Leather', 'Imported', 'synthetic lining', 'F...","43,930inClothing,Shoesamp;Jewelry(",5 star,B00007GDFV,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,['Authentic crunch leather with rich floral em...,16.95,"[B07C9V84JD, B01J6JE05G, B07J11WZ5Y, B07JJQFHS...","[B07C9V84JD, B01J6JE05G, B07JJQFHS5, B003EGITU...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN,female,Leather,Accessories
795,Disguise Tiny Treats Pink Leopard,NaN,"['polyester', 'You can return this item for an...","5,843,505inClothing,ShoesJewelry(",5 star,B0002C6NL6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,['A grrreat pink jumpsuit with attached tail a...,25.99,NaN,NaN,NaN,NaN,NaN,NaN,female,Polyester,Dresses/Jumpsuits


In [6]:
df[(df["brand"] == "Holloway") & (df["price"] == 36.825)] 

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1,gender,material,category
1555,Holloway Olympian Nylon Pullover Jacket From S...,Holloway,nan,"3,082,499inSportsOutdoors(",NaN,B000C18C5W,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,['¶Duraweav™ Oxford Nylon Shell¶Wind and water...,36.825,NaN,NaN,NaN,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN,unisex,Nylon,Outerwear


In [4]:
df.shape

(1262, 19)

## Building Dataset

In [5]:
documents = []
for index, row in df.iterrows():
    descriptions = row["description"].strip("[]").strip("''")

    doc = Document(
        content = f"{row['title']}\n {descriptions}",
        meta = {
            'asin': row['asin'],
            'title': row['title'],
            'brand': row['brand'],
            'price': row['price'],
            'gender': row['gender'],
            'material': row['material'],
            'category': row['category'],
        }
    )
    documents.append(doc)

In [6]:
documents[:5]

[Document(id=bd174fb86972d98caf70c4784fb24ef902812b76b05fde64b1b9c3874702f92c, content: 'Buxton Heiress Pik-Me-Up Framed Case
  Authentic crunch leather with rich floral embossed logo heires...', meta: {'asin': 'B00007GDFV', 'title': 'Buxton Heiress Pik-Me-Up Framed Case', 'brand': 'Buxton', 'price': 16.95, 'gender': 'female', 'material': 'Leather', 'category': 'Accessories'}),
 Document(id=373b8364eeb48825409004f1f3f0b765addf65373eb76d8c6b11075d24a61b89, content: 'Disguise Tiny Treats Pink Leopard
  A grrreat pink jumpsuit with attached tail and character hood wit...', meta: {'asin': 'B0002C6NL6', 'title': 'Disguise Tiny Treats Pink Leopard', 'brand': nan, 'price': 25.99, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}),
 Document(id=b86b8f98325f7d9b8387509cea143775f303bf7c76c6d5bf63ee37e087cfe177, content: 'Dream PJ's Blue - Large - Part #: 25BLG
  SOFT AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT PASTEL COLORS ...', meta: {'asin': 'B0002TKBSU', 'title': "Dre

## Membuat Storing Pipeline

In [4]:

os.environ["MONGO_CONNECTION_STRING"] = getpass("Masukkan MongoDB Connection String Anda: ")

In [8]:
pipeline_storing = Pipeline()

In [9]:
document_store = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="products",
    vector_search_index="vector_index",
    full_text_search_index="search_index",
)

In [ ]:
pipeline = Pipeline()
pipeline.add_component("embedder",SentenceTransformersDocumentEmbedder())
pipeline.add_component("writer",DocumentWriter(document_store=document_store,policy=DuplicatePolicy.OVERWRITE))
pipeline.connect("embedder","writer")

🚅 Components
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - embedder.documents -> writer.documents (List[Document])

In [9]:
pipeline.run({
    "embedder":{
        "documents":documents
    }
})

d:\Enviroments\anaconda3\envs\day45-haystack\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rasyi\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fal

{'writer': {'documents_written': 1262}}

## Menyimpan Category dan Material di koleksi 

In [5]:
client = MongoClient(os.environ['MONGO_CONNECTION_STRING'])
db = client.depato_store
material_collection = db.materials
category_collection = db.categories

In [10]:
materials = df['material'].unique().tolist()
categories = df['category'].unique().tolist()
materials

['Leather',
 'Polyester',
 'Unknown',
 'Cotton',
 'Nylon',
 'Wood',
 'Synthetic',
 'Others',
 'Metal',
 'Fleece',
 'Silicone',
 'Gemstone',
 'Vinyl',
 'Viscose',
 'Feather',
 'Velvet',
 'Silk',
 'Textile',
 'Olefin',
 'Acrylic',
 'Chiffon',
 'Wool',
 'Canvas',
 'Straw',
 'Suede',
 'Foam',
 'Microfiber',
 'Rayon',
 'Denim',
 'Elastane/Spandex']

In [11]:
documents_material= [ {"name":m} for m in materials]
documents_category = [ {"name":c} for c in categories]

In [ ]:
material_collection.insert_many(documents_material)
category_collection.insert_many(documents_category)

InsertManyResult([ObjectId('691277f84fc944e355f6628c'), ObjectId('691277f84fc944e355f6628d'), ObjectId('691277f84fc944e355f6628e'), ObjectId('691277f84fc944e355f6628f'), ObjectId('691277f84fc944e355f66290'), ObjectId('691277f84fc944e355f66291'), ObjectId('691277f84fc944e355f66292'), ObjectId('691277f84fc944e355f66293'), ObjectId('691277f84fc944e355f66294'), ObjectId('691277f84fc944e355f66295')], acknowledged=True)

: 

# Insert Common Policy

In [6]:
with open("../data/commont information/common_information_ecommerce.txt" , "r") as f:
     texts = f.read()
texts

'Common Information for E-Commerce\n\n1.  Return & Refund Policy\n\n-   Return Period: Barang dapat dikembalikan dalam waktu 7–14 hari\n    setelah diterima.\n-   Conditions: Produk harus dalam kondisi asli, belum digunakan, dan\n    dengan kemasan lengkap.\n-   Refund Method: Pengembalian dana dilakukan melalui metode pembayaran\n    awal (bank transfer, e-wallet, atau kartu kredit).\n-   Non-Returnable Items: Produk digital, pakaian dalam, dan produk yang\n    sudah digunakan tidak dapat dikembalikan.\n-   Processing Time: Proses refund memakan waktu 3–5 hari kerja setelah\n    barang diterima dan diverifikasi.\n\n2.  Shipping Information\n\n-   Shipping Partners: JNE, J&T Express, SiCepat, Pos Indonesia, Ninja\n    Xpress.\n-   Estimated Delivery Time: Jabodetabek: 1–3 hari kerja, Luar Jawa: 3–7\n    hari kerja.\n-   Shipping Fee: Dihitung otomatis berdasarkan berat dan lokasi\n    pengiriman.\n-   Tracking: Nomor resi akan dikirim melalui email/SMS setelah pesanan\n    dikonfirmasi

In [7]:

doc = Document(content=texts)
splitter = DocumentSplitter(split_by="word", split_length=100, split_overlap=30)
result = splitter.run(documents=[doc])
documents = result["documents"]

In [10]:
db.create_collection("common_information")

Collection(Database(MongoClient(host=['ac-ictdpnb-shard-00-00.ozynzme.mongodb.net:27017', 'ac-ictdpnb-shard-00-01.ozynzme.mongodb.net:27017', 'ac-ictdpnb-shard-00-02.ozynzme.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-3rwbyh-shard-0', tls=True), 'depato_store'), 'common_information')

In [11]:
document_store = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="common_information",
    vector_search_index="vector_index",
    full_text_search_index="search_index",
)

In [12]:
query_pipeline = Pipeline()
query_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder())
query_pipeline.add_component("writter" , DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE))
query_pipeline.connect("embedder", "writter")

🚅 Components
  - embedder: SentenceTransformersDocumentEmbedder
  - writter: DocumentWriter
🛤️ Connections
  - embedder.documents -> writter.documents (List[Document])

In [13]:
query_pipeline.run({
    "embedder":{
        "documents" : documents
    }
})

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.74s/it]


{'writter': {'documents_written': 6}}